In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import cohere
from dotenv import load_dotenv
import os
import nltk
nltk.download('punkt')  # Download the tokenizer models

from nltk.tokenize import word_tokenize
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yunethirikhin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from the environment
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
load_dotenv()

# Retrieve the API key from the environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


client_gpt = OpenAI(
  api_key=OPENAI_API_KEY,  # this is also the default, it can be omitted
)

In [3]:
co = cohere.ClientV2(COHERE_API_KEY)
client = QdrantClient(url="http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='aapl_10k_10q_forms'), CollectionDescription(name='earnings_calls'), CollectionDescription(name='financial_news')])

In [4]:
collection_name = "earnings_calls"
collection_info = client.get_collection(collection_name)

print(f"Collection info for {collection_name}:")
print(collection_info)

Collection info for earnings_calls:
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=48 segments_count=4 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segment

In [4]:
client.create_collection(
    collection_name="summarizer_test3",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)

True

In [ ]:
file_path = '/home/sarahgohrazer/Projects/Stocks-MultiAgent/aapl_10k_forms/apple_filings_text/000032019324000123-aapl-20240928.txt' 
with open(file_path, 'r') as file:
    text = file.read()

# Divide the text into chunks
chunk_size = 3000  # Adjust this size as needed
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [47]:
model="embed-english-v3.0"

doc_embeddings = co.embed(texts=chunks,
                          model=model,
                          input_type="search_document",
                          embedding_types=['float'])

NameError: name 'chunks' is not defined

In [5]:
query ="""
Provide a summary of the 10K report for Apple in 2024
"""

In [22]:
points = []
for idx, (embedding, doc) in enumerate(zip(doc_embeddings.embeddings.float_, chunks)):
    point = PointStruct(
        id=idx,
        vector=embedding,
        payload={"document": doc}
    )
    points.append(point)

In [11]:
operation_info = client.upsert(
    collection_name="summarizer_test3",
    points=points
)

In [6]:
model="embed-english-v3.0"

query_embeddings = co.embed(texts=[query],
                          model=model,
                          input_type="search_query",
                          embedding_types=['float'])

In [10]:
search_result = client.query_points(
    collection_name="aapl_10k_10q_forms", query=query_embeddings.embeddings.float_[0], limit=10
).points

In [12]:
for hit in search_result:
    print(hit.payload.keys())
    print(f"ID: {hit.id}, Document: {hit.payload['document']}, Score: {hit.score}")

dict_keys(['document', 'file_name', 'chunk_index'])
ID: 232703908057093973, Document: basis, for the Company, the S&P 500 Index and the Dow Jones U.S. Technology Supersector Index. The graph assumes $100 was invested in each of the Company’s common stock, the S&P 500 Index and the Dow Jones U.S. Technology Supersector Index as of the market close on September 27, 2019. Past stock price performance is not necessarily indicative of future stock price performance.
September 2019
September 2020
September 2021
September 2022
September 2023
September 2024
Apple Inc.
$
100
$
207
$
273
$
281
$
322
$
430
S&P 500 Index
$
100
$
113
$
156
$
131
$
155
$
210
Dow Jones U.S. Technology Supersector Index
$
100
$
146
$
216
$
156
$
215
$
322
Item 6.    [Reserved]
Apple Inc. | 2024 Form 10-K | 20
Item 7.    Management’s Discussion and Analysis of Financial Condition and Results of Operations
The following discussion should be read in conjunction with the consolidated financial statements and accompanying 

In [13]:
document_list = [point.payload['document'] for point in search_result]

document_list

['basis, for the Company, the S&P 500 Index and the Dow Jones U.S. Technology Supersector Index. The graph assumes $100 was invested in each of the Company’s common stock, the S&P 500 Index and the Dow Jones U.S. Technology Supersector Index as of the market close on September\xa027, 2019. Past stock price performance is not necessarily indicative of future stock price performance.\nSeptember 2019\nSeptember 2020\nSeptember 2021\nSeptember 2022\nSeptember 2023\nSeptember 2024\nApple Inc.\n$\n100\n$\n207\n$\n273\n$\n281\n$\n322\n$\n430\nS&P 500 Index\n$\n100\n$\n113\n$\n156\n$\n131\n$\n155\n$\n210\nDow Jones U.S. Technology Supersector Index\n$\n100\n$\n146\n$\n216\n$\n156\n$\n215\n$\n322\nItem 6.\xa0\xa0\xa0\xa0[Reserved]\nApple Inc. | 2024 Form 10-K | 20\nItem 7.\xa0\xa0\xa0\xa0Management’s Discussion and Analysis of Financial Condition and Results of Operations\nThe following discussion should be read in conjunction with the consolidated financial statements and accompanying notes in

checking token limit, max 4096 tokens in each document

In [18]:
# Assuming document_list contains text-based documents (strings)
tokens_per_document = [len(word_tokenize(doc)) for doc in document_list]

# Printing the number of tokens for each document
print(tokens_per_document)

## max 4096 tokens in each document 

[588, 535, 129, 585, 500, 596, 556, 498, 588, 564]


In [17]:
rerank_results = co.rerank(
    model="rerank-english-v3.0",
    query=query,
    documents=document_list,
    top_n=3,
    return_documents=True
)

In [17]:
# print(dir(rerank_results))
top_documents = [item.document.text for item in rerank_results.results]
print(top_documents)
print(len(top_documents))

NameError: name 'rerank_results' is not defined

send to open ai for query

In [20]:
# Combine the top documents into a single prompt
combined_documents = "\n\n".join(document_list[:3])  # Use top 3 or however many you want

# Prepare the prompt for the combined documents
prompt = f"Query: {query}\n\nDocuments: {combined_documents}\n\nAnswer:"

# Generate a response using the OpenAI GPT model
response = client_gpt.responses.create(
    model="gpt-3.5-turbo",  # Use the desired OpenAI model
    input=prompt,  # The prompt for the LLM
)

# Output the answer
print(response.output_text.strip())


AttributeError: 'OpenAI' object has no attribute 'create'